In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.graph_objects as go


players = pd.read_csv('Player.csv')
player_atts = pd.read_csv('Player_Attributes.csv')
teams = pd.read_csv('Team.csv')
matches = pd.read_csv('Match.csv')
leagues = pd.read_csv('League.csv')

player_atts['potential_rating_ratio'] = ((player_atts['potential'] / player_atts['overall_rating']) * 100)

In [ ]:
teams[teams['team_long_name']=='RCD Espanyol']

In [ ]:
matches['date'] = pd.to_datetime(matches['date'])
rcde_matches = matches[matches['home_team_api_id']== 8558]

In [ ]:
for column in rcde_matches:
    if 'home_player' in column:
        rcde_matches[column] = rcde_matches[column].astype('Int64')
        rcde_matches = rcde_matches.merge(players, left_on=column, right_on='player_api_id', how='left', suffixes=('','_new'))

In [ ]:
rcde_names = pd.unique(rcde_matches['player_name_new'].values.ravel()).tolist()
rcde_players = players[players['player_name'].isin(rcde_names)]


In [ ]:
rcde_player_atts = player_atts[player_atts['player_api_id'].isin(rcde_players['player_api_id'])]

In [ ]:
# add name into the player attributes df 
rcde_player_atts = rcde_player_atts.merge(rcde_players[['player_api_id', 'player_name']], on='player_api_id', how='left')

In [ ]:
# reorder columns
cols = rcde_player_atts.columns.tolist()
cols_to_move = ['player_name', 'potential_rating_ratio']
target_col = 'player_api_id'

for i in cols_to_move:
    cols.remove(i)
cols.insert(cols.index(target_col) + 1, cols_to_move[0])
cols.insert(cols.index(target_col) + 2, cols_to_move[1])

rcde_player_atts = rcde_player_atts[cols]

In [ ]:
rcde_player_atts.sort_values('potential_rating_ratio', ascending=False, inplace=True)

In [58]:
rcde_player_atts

,id,player_fifa_api_id,player_api_id,player_name,potential_rating_ratio,date,overall_rating,potential,preferred_foot,attacking_work_rate,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1501,143372,189242,184536,Philippe Coutinho,151.785714,2008-08-30 00:00:00,56.0,85.0,right,medium,...,80.0,56.0,25.0,33.0,31.0,12.0,7.0,9.0,14.0,6.0
1502,143373,189242,184536,Philippe Coutinho,151.785714,2007-02-22 00:00:00,56.0,85.0,right,medium,...,80.0,56.0,25.0,33.0,31.0,12.0,7.0,9.0,14.0,6.0
1085,90432,183969,121716,Juan Forlin,147.169811,2007-02-22 00:00:00,53.0,78.0,right,high,...,71.0,38.0,50.0,57.0,81.0,1.0,21.0,34.0,21.0,21.0
1084,90431,183969,121716,Juan Forlin,147.169811,2008-08-30 00:00:00,53.0,78.0,right,high,...,71.0,38.0,50.0,57.0,81.0,1.0,21.0,34.0,21.0,21.0
610,58381,161863,45440,Francisco Chica,143.137255,2007-02-22 00:00:00,51.0,73.0,left,medium,...,70.0,55.0,49.0,53.0,70.0,14.0,11.0,41.0,14.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,71317,110371,24099,Hurtado Perez Moises,NaN,2007-02-22 00:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
768,73790,146577,75194,Ivan Alonso,NaN,2007-02-22 00:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1119,92889,52401,43329,Kalu Uche,NaN,2007-02-22 00:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,109763,173179,37879,Marc Torrejon Moya,NaN,2007-02-22 00:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
#attribute_cols = rcde_player_atts.select_dtypes(include='number').columns.difference(['player_api_id', 'player_fifa_api_id', 'date', 'id'])
id_cols = ['player_api_id', 'player_fifa_api_id', 'date', 'id']
attribute_cols = rcde_player_atts.select_dtypes(include='number').columns.difference(id_cols + ['potential_rating_ratio']).tolist()
attribute_cols = ['potential_rating_ratio'] + attribute_cols

pivot_df = rcde_player_atts.pivot_table(index=None, columns='player_name', values=attribute_cols, aggfunc='mean')
pivot_df = pivot_df.T 

In [75]:
import plotly.io as pio

pio.renderers.default = "notebook_connected"

# Interactive Heatmap with Plotly
fig = go.Figure(
    data=go.Heatmap(
        z=pivot_df.values,
        x=pivot_df.columns,       # Player Names on X-axis
        y=pivot_df.index,          # Attributes as row labels (text)
        colorscale='Greens',       # Green colorscale
        colorbar=dict(title='Attribute Value'),
        hovertemplate='Player: %{x}<br>Attribute: %{y}<br>Value: %{z}<extra></extra>'
    )
)

# Layout adjustments
fig.update_layout(
    title='Player Attributes',
    xaxis=dict(title='Attributes', tickangle=45),
    yaxis=dict(title='Player Names'),
    autosize=True,
    height=600,
    width=1200
)

# Show interactive plot with zoom & pan
fig.show()

